In [14]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from ray import tune

import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np

In [15]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 2
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [59]:
model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT)
print(model.classifier[1])
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
print(model.classifier[1])

Linear(in_features=1280, out_features=1000, bias=True)
Linear(in_features=1280, out_features=2, bias=True)


In [17]:
# tirar essa parte, porque não queremos testar redes diferentes

# def define_model(trial):
#     # We optimize the number of layers, hidden units and dropout ratio in each layer.
#     n_layers = trial.suggest_int("n_layers", 1, 3) # number of layers
#     layers = []

#     in_features = 1280
#     for i in range(n_layers):
#         out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128) # hidden units
#         layers.append(nn.Linear(in_features, out_features))
#         layers.append(nn.ReLU())
#         p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5) # dropout ratio
#         layers.append(nn.Dropout(p))

#         in_features = out_features
#     layers.append(nn.Linear(in_features, CLASSES))
#     layers.append(nn.LogSoftmax(dim=1))

#     return nn.Sequential(*layers)

In [18]:
from torchvision import transforms as T
img_size = 224

transform = T.Compose([
                T.Resize((img_size,img_size)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [19]:
from torch.utils.data import DataLoader
from configs import Inputs
from utils.augmentations import get_transforms
from utils.data import FullRadiographSexDataset

val_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                       fold_nums=Inputs().val_folds,
                                       transforms=get_transforms(Inputs(), subset=["train"]))

val_dataloader = DataLoader(val_dataset,
                            batch_size=2,
                            shuffle=False,
                            num_workers=0)

train_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                         fold_nums=Inputs().train_folds,
                                         transforms=get_transforms(Inputs(), subset=["train"]))

train_dataloader = DataLoader(train_dataset,
                              batch_size=2,
                              shuffle=False,
                              num_workers=0)

Using only horizontal flip augmentation.
Using only horizontal flip augmentation.


In [49]:
config = {
    "lr": tune.loguniform(1e-5, 1e-1),
    "optimizer_name": tune.choice(["Adam", "RMSprop", "SGD"])
}

In [62]:
def objective(config):

    # Gerar o modelo
    #model = define_model(trial).to(DEVICE)
    model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model = model.to(DEVICE)

    # Gerar optimizer
    optimizer_name = config['optimizer_name'].sample()
    lr = config["lr"].sample()

    print("opt_name:", optimizer_name, "\nlr:", lr)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = train_dataloader, val_dataloader

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        with tune.checkpoint_dir(epoch) as checkpoin_dir:
            path = os.path.join(checkpoin_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss, accuracy=accuracy)

    return accuracy

In [63]:
objective(config)

opt_name: SGD 
lr: 5.483272075747051e-05


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [2, 691200]